In [ ]:
# Chapter 11

Examples and Exercises from Think Stats, 2nd Edition

http://thinkstats2.com

Copyright 2016 Allen B. Downey

MIT License: https://opensource.org/licenses/MIT

In [14]:
from os.path import basename, exists


def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + local)


download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/thinkstats2.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/thinkplot.py")


import numpy as np
import pandas as pd

import thinkstats2
import thinkplot

download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/nsfg.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/first.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemResp.dct")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemResp.dat.gz")

download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemPreg.dct")
download(
    "https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemPreg.dat.gz"
)

import first

live, firsts, others = first.MakeFrames()

import statsmodels.formula.api as smf

import nsfg

live = live[live.prglngth>30]
resp = nsfg.ReadFemResp()
resp.index = resp.caseid
join = live.join(resp, on='caseid', rsuffix='_r')
join.shape

(8884, 3331)

In [ ]:
**Exercise:11.1 ** Suppose one of your co-workers is expecting a baby and you are participating in an office pool to predict the date of birth. Assuming that bets are placed during the 30th week of pregnancy, what variables could you use to make the best prediction? You should limit yourself to variables that are known before the birth, and likely to be available to the people in the pool.

In [16]:
import first
live, firsts, others = first.MakeFrames()
live = live[live.prglngth>30]

In [17]:
model = smf.ols('prglngth ~ birthord==1', data=live)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               prglngth   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     9.475
Date:                Tue, 07 Feb 2023   Prob (F-statistic):            0.00209
Time:                        15:53:34   Log-Likelihood:                -18294.
No. Observations:                8884   AIC:                         3.659e+04
Df Residuals:                    8882   BIC:                         3.661e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                38.8184      0.028   1389.249      0.000      38.764      38.873
birthord == 1[T.True]     0.1240      0.040      3.078      0.002       0.045       0.203
==============================================================================
Omnibus:                     1606.695   Durbin-Watson:                   1.628
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6207.939
Skew:                          -0.863   Prob(JB):                         0.00
Kurtosis:                       6.713   Cond. No.                         2.58
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
**Exercise:11.3** If the quantity you want to predict is a count, you can use Poisson regression, which is implemented in StatsModels with a function called poisson. It works the same way as ols and logit. As an exercise, let’s use it to predict how many children a woman has born; in the NSFG dataset, this variable is called numbabes.
Suppose you meet a woman who is 35 years old, black, and a college graduate whose annual household income exceeds $75,000. How many children would you predict she has born?

In [18]:
join.numbabes.replace()
formula='numbabes ~ ager + C(race) + totincr + educat'
model = smf.poisson(formula, data=join)
results = model.fit()
results.summary() 

Optimization terminated successfully.
         Current function value: 1.687055
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:               numbabes   No. Observations:                 8884
Model:                        Poisson   Df Residuals:                     8878
Method:                           MLE   Df Model:                            5
Date:                Tue, 07 Feb 2023   Pseudo R-squ.:                 0.03109
Time:                        15:53:38   Log-Likelihood:                -14988.
converged:                       True   LL-Null:                       -15469.
Covariance Type:            nonrobust   LLR p-value:                1.106e-205
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        1.0842      0.045     23.995      0.000       0.996       1.173
C(race)[T.2]    -0.1398      0.015     -9.464      0.000      -0.169      -0.111
C(race)[T.3]    -0.0914      0.025     -3.717      0.000      -0.140      -0.043
ager             0.0208      0.001     20.474      0.000       0.019       0.023
totincr         -0.0179      0.002     -9.442      0.000      -0.022      -0.014
educat          -0.0443      0.003    -15.139      0.000      -0.050      -0.039
================================================================================
"""

In [ ]:
Now we can predict the number of children for a woman who is 35 years old, black, and a college
graduate whose annual household income exceeds $75,000

In [20]:
columns = ['ager', 'race', 'totincr', 'educat']
result = pd.DataFrame([[35, 1, 14, 16]], columns=columns)
results.predict(result)

0    2.342182
dtype: float64

In [ ]:
**Exercise: 11.4 ** If the quantity you want to predict is categorical, you can use multinomial logistic regression, which is implemented in StatsModels with a function called `mnlogit`. As an exercise, let’s use it to guess whether a woman is married, cohabitating, widowed, divorced, separated, or never married; in the NSFG dataset, marital status is encoded in a variable called `rmarital`.

Suppose you meet a woman who is 25 years old, white, and a high school graduate whose annual household income is about $45,000. What is the probability that she is married, cohabitating, etc?

In [21]:
formula_mlg ='rmarital ~ ager + C(race) + totincr + educat'
model = smf.mnlogit(formula_mlg, data=join)
results = model.fit()
results.summary() 

Optimization terminated successfully.
         Current function value: 1.087603
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:               rmarital   No. Observations:                 8884
Model:                        MNLogit   Df Residuals:                     8854
Method:                           MLE   Df Model:                           25
Date:                Tue, 07 Feb 2023   Pseudo R-squ.:                  0.1655
Time:                        15:54:36   Log-Likelihood:                -9662.3
converged:                       True   LL-Null:                       -11579.
Covariance Type:            nonrobust   LLR p-value:                     0.000
================================================================================
  rmarital=2       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        4.4532      0.279     15.977      0.000       3.907       5.000
C(race)[T.2]    -0.9219      0.089    -10.409      0.000      -1.095      -0.748
C(race)[T.3]    -0.6334      0.136     -4.674      0.000      -0.899      -0.368
ager            -0.0570      0.006     -9.754      0.000      -0.068      -0.046
totincr         -0.1302      0.012    -11.298      0.000      -0.153      -0.108
educat          -0.2051      0.019    -11.017      0.000      -0.242      -0.169
--------------------------------------------------------------------------------
  rmarital=3       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -4.5432      0.916     -4.960      0.000      -6.338      -2.748
C(race)[T.2]    -0.4405      0.236     -1.865      0.062      -0.904       0.023
C(race)[T.3]     0.0329      0.335      0.098      0.922      -0.623       0.689
ager             0.1301      0.019      6.833      0.000       0.093       0.167
totincr         -0.3251      0.032    -10.182      0.000      -0.388      -0.263
educat          -0.1010      0.048     -2.100      0.036      -0.195      -0.007
--------------------------------------------------------------------------------
  rmarital=4       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -2.6485      0.302     -8.783      0.000      -3.239      -2.057
C(race)[T.2]    -0.3188      0.093     -3.417      0.001      -0.502      -0.136
C(race)[T.3]    -0.7664      0.171     -4.485      0.000      -1.101      -0.431
ager             0.0641      0.007      9.817      0.000       0.051       0.077
totincr         -0.2270      0.012    -19.588      0.000      -0.250      -0.204
educat           0.0673      0.017      4.036      0.000       0.035       0.100
--------------------------------------------------------------------------------
  rmarital=5       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.8231      0.345      2.385      0.017       0.147       1.500
C(race)[T.2]    -1.0491      0.104    -10.125      0.000      -1.252      -0.846
C(race)[T.3]    -0.5644      0.156     -3.624      0.000      -0.870      -0.259
ager             0.0063      0.007      0.872      0.383      -0.008       0.020
totincr         -0.2923      0.015    -20.149      0.000      -0.321      -0.264
educat          -0.0136      0.021     -0.639      0.523      -0.056       0.028
--------------------------------------------------------------------------------
  rmarital=6       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        6.2370      0.275     22.654      0.000       5.697       6.777
C

In [22]:
Make a prediction for a woman who is 25 years old, white, and a high
school graduate whose annual household income is about $45,000.

SyntaxError: invalid syntax (351000821.py, line 1)

In [23]:
columns = ['ager', 'race', 'totincr', 'educat']
result = pd.DataFrame([[25, 2, 11, 12]], columns=columns)
results.predict(result)

,0,1,2,3,4,5
0,0.748384,0.125474,0.001103,0.035295,0.023813,0.065931
